In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('novel_story_sep.csv', lineterminator='\n')

In [3]:
df.columns

Index(['itemId', 'title', 'pubDate', 'coverSmallUrl', 'coverLargeUrl',
       'publisher', 'customerReviewRank', 'author', 'isbn', 'link', 'story',
       'story_sep'],
      dtype='object')

https://happy-obok.tistory.com/5

### LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습하기

0. gensim 라이브러리 설치 : 자연어를 벡터로 변환하는데 필요한 대부분의 편의 기능을 제공하고 있는 라이브러리 (Word2vec 포함)

In [4]:
# !pip install gensim==3.8.3

In [5]:
import gensim
gensim.__version__

'3.8.3'

1. gensim.models.wrappers.LdaMallet 모듈을 사용해서 LDA 모델 개수 추정
- https://radimrehurek.com/gensim_3.8.3/models/wrappers/ldamallet.html

In [6]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet

1) LDA 모델에 들어갈 객체를 만든다.
- id2word : dictionary 에 list of list of str 형식의 documents를 입력하면 Dictionary가 학습됨. 전체 말뭉치에 단어가 겹치지 않도록 하나씩 dictionary에 저장됨.
- corpus : 단어들을 bag-of-words 형태에서 list if (token_id, token_count) 2-tuples로 변환한다.


= [[(id2 word [id], freq) for id, freq in cp] for cp in corpus [:50]]

In [7]:
stories = df['story_sep']

In [8]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [9]:
id2word = corpora.Dictionary(data_word)

In [10]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../감성사전/mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=18, random_seed=10, id2word=id2word)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print(coherence_ldamallet) #응집성 지수

Mallet LDA: 18 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 336
total tokens: 82218
<10> LL/token: -9.29464
<20> LL/token: -8.75692
<30> LL/token: -8.54031
<40> LL/token: -8.44545

0	2.77778	남자 다시 소년 죽음 가족 감정 여섯 묘사 주인공 펼쳐진다 사람 이었던 않고 사건 없는 여덟 편집 하게 그동안 뜨거운 
1	2.77778	우리 여성 이름 역사 결혼 스스로 여자 누구 서로 사랑 가슴 욕망 타인 만나 세상 언어 이번 nan 슬픔 시작 
2	2.77778	자신 그녀 기억 과거 존재 남편 결국 시작 아내 상처 일본 사실 사람 없는 하지 하게 현대문학 진실 상황 함께 
3	2.77778	세계 독자 시작 되는 젊은 한국 시리즈 새로운 여름 진실 통해 시점 이미 국문학 이상 발표 지난 내면 선사 장르 
4	2.77778	한국 가장 최고 그려 당시 어머니 편의 가는 복수 처음 만큼 특히 아름다운 완성 대표 했다 예술 되어 연구 전개 
5	2.77778	사건 가족 엄마 없이 미스터리 스릴러 생활 사는 없는 살아가는 살인 받은 반전 남자 심리 의사 새롭게 가시노 시선 만남 
6	2.77778	문학상 선정 하

0.3633254837536719


In [11]:
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, random_seed=10, id2word=id2word)
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print(coherence_ldamallet) #응집성 지수

---

### 토픽 수 별로 일관성 점수를 계산해서 가장 좋은 토픽 수의 모델 찾기

models.coherencemodel 주제 모델에 대한 주제 일관성을 계산

* model :주제가 제공되지 않은 경우 사전 훈련된 주제 모델을 제공해야 합니다. 현재 지원 LdaModel, LdaMulticore, LdaMallet와 LdaVowpalWabbit.
* topics(list of list of str, optional) :토큰 화 된 토픽의 목록
* texts (list of list of str, optional) :슬라이딩 창 기반 (예 : coherence =c_something) 확률 추정 기를 사용하는 일관성 모델에 필요한 토큰 화 된 텍스트.
* corpus (iterable of list of (int, number), optional) :BoW 형식의 코퍼스.
* dictionary (Dictionary, optional) : Gensim dictionary mapping of id word to create corpus. If model.id2 word is present, this is not needed. If both are provided, passed dictionary will be used.
* coherence ({'u_mass', 'c_v', 'c_uci', 'c_npmi'}, optional) :
* topn (int, optional) : 각 주제에서 추출할 최상위 단어 수에 해당하는 정수
* processes (int, optional) : 확률 추정 단계에 사용할 프로세스 수

#### <Coherence 점수를 계산하여 좋은 LDA 모델 찾기>

In [12]:
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# coherence_ldamallet #응집성 지수

In [13]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [14]:
# Can take a long time to run.
limit=18; start=6; step=3;
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=start, limit=limit, step=step)

In [15]:
# x = range(start, limit, step)
# topic_num = 0
# count = 0
# max_coherence = 0
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", cv)
#     coherence = cv
#     if coherence >= max_coherence:
#         max_coherence = coherence
#         topic_num = m
#         model_list_num = count   
#     count = count+1

### 모델 계산시간이 오래걸려 그냥 지정된 모델로 지정

In [16]:
# Select the model and print the topics
# optimal_model = model_list[model_list_num]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics()[5])

(5, '0.038*"엄마" + 0.032*"가족" + 0.020*"일본" + 0.019*"작은" + 0.017*"동안" + 0.017*"서로" + 0.017*"일상" + 0.016*"함께" + 0.014*"아빠" + 0.013*"평범한"')


In [17]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data['itemId'],Data['title'],Data['story_sep'], sent_topics_df], axis=1)

    return(sent_topics_df)

In [18]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df)

/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_43651/2572131905.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/hr/y8znbyhd4058w4d5l14qqsv80000gn/T/ipykernel_43651/2572131905.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


<class 'pandas.core.frame.DataFrame'>


In [19]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [20]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [25]:
sent_topics_sorteddf_mallet

,itemId,title,story_sep,Dominant_Topic,Perc_Contribution,Topic_Keywords
988,277678422,화산전생 8,무당 전생 기적 앱스토어 정준 신무협 영웅 동경 사내 회귀 훗날 찬란히 빛날 절대자...,0,0.5131,"사랑, 남자, 없는, 죽음, 시절, 다른, 어린, 다시, 운명, 그것"
664,350529358,고구려 3: 낙랑정벌,대사 고조선 고구려 몽땅 중국 역사 쓸어 담는 동북공정 허구 깨는 웅혼 고구려 역사...,1,0.7291,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
551,251651152,홀(The Hole),이미 뚫려 있던 구멍 실체 마주 하다 편혜영 세계 봄호 통해 발표 식물 애호 시작 ...,2,0.4481,"자신, 기억, 아버지, 남편, 아들, 함께, 하게, 과정, 시작, 아내"
451,252098561,제7회 젊은작가상 수상작품집(2016),성실하고 활발하게 자신 세계 조해 가는 일곱 명의 젊은 젊은 수상 작품집 제정 젊은...,3,0.4226,"세계, 통해, 편의, 한국, 문학상, 이번, 독자, 소설가, 발표, 젊은"
487,237256879,구르미 그린 달빛 2: 달무리,웹소설 전설 윤이수 로맨스 윤이수 구르 그린 달빛 달무리 박보검 김유정 주연 월화드...,4,0.6075,"그린, 인물, 한국, 조선, 구성, 여인, 배경, 가는, 이름, 복수"
602,349598232,토와의 정원,있다는 굉장한 일이구 두운 심연 잠겨 올려다보는 맑고 투명한 달팽이 식당 츠바키 문...,5,0.6008,"엄마, 가족, 일본, 작은, 동안, 서로, 일상, 함께, 아빠, 평범한"
532,340165107,태백산맥 1: 제1부 한의 모닥불,한반도 분단 전쟁 제대로 그려 오늘 역사 이어주고 태백산맥 전남 여수 주둔 있던 국...,6,0.8690,"선정, 보다, 하여, 계절, 독자, 세력, 벌교, 겨울, 되어, 젊은"
227,354469147,전지적 독자 시점 PART 1: 1,토털 웹소설 현재 진행형 레전드 마침내 단행본 만나는 전지 독자 시점 피아 누적 판...,7,0.7500,"독자, 드라마, 영화, 세계, 하나, 제작, 되는, 시점, 이미, 연재"
482,235804015,개선문 1,사랑 도피 사치 아닌 평화 안전 기쁨 축제 였던 불안 시대 리다 민음사 세계문학 전...,8,0.6959,"인간, 도시, 인류, 하기, 새로운, 고양이, 그려, 대표, 프랑스, 되고"
641,255619017,풀꽃도 꽃이다 1,정글만리 펴낸 조정래 신작 우리 사회 교육 지향 제안 하는 조정래 풀꽃 꿈틀 거대한...,9,0.5133,"아이, 시간, 친구, 자신, 학교, 같은, 없는, 고민, 할머니, 위로"


In [22]:
df_topic_story

,itemId,title,story_sep,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,281536876,파친코 1,어디 하지 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 이민 진의 파친코 면서 끝...,1,0.2880,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
1,281536885,파친코 2,어디 하지 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 이민 진의 파친코 면서 끝...,1,0.2919,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
2,348921505,불편한 편의점(40만부 기념 벚꽃 에디션),인터넷 한정 특별판 매장 구매 바로 드림 구매 다른 표지 에디 제공 됩니다 플러스 ...,5,0.1765,"엄마, 가족, 일본, 작은, 동안, 서로, 일상, 함께, 아빠, 평범한"
3,354595636,저주토끼,부커상 최종 후보 명작 한국 호러 판타지 대표 정보라 대표 인터내셔널 부커상 후보 ...,12,0.1830,"사람, 위해, 세상, 선택, 하나, 했다, 없는, 당신, 생각, 모두"
4,266107628,아몬드,괴물 다른 괴물 만났다 영화 같은 강렬한 사건 매혹 문체 시선 사로잡는 한국 어덜트...,15,0.2229,"마음, 청소년, 소년, 자신, 소녀, 누구, 감동, 상처, 세상, 인생"
...,...,...,...,...,...,...
995,288668147,하얀 늑대들 3,가진 용기 입담 농부 카셀 위대한 기사 캡틴 되기까지 처음 지난 지금 전자책 올라있...,16,0.3189,"시작, 그녀, 같은, 판타지, 하는데, 게임, 다시, 주인공, 우연히, 있을까"
996,289078215,네버무어: 모리건 크로우와 원드러스 평가전 1,불행 태어난 저주받은 아이 운명 거부 신비 도시 무어 떠나다 제시카 타운센드 무려 ...,9,0.1724,"아이, 시간, 친구, 자신, 학교, 같은, 없는, 고민, 할머니, 위로"
997,243131474,춘분 지나고까지,세기 문호 소세키 정수 단단한 번역 꼼꼼한 편집 디자인 새롭게 읽는 소세키 전집 춘...,10,0.2073,"사람, 위해, 우리, 자신, 대한, 되어, 가장, 통해, 욕망, 그려"
998,268911218,별의 계승자 2: 가니메데의 친절한 거인,가니메데 거인 돌아왔다 다시 미궁 빠진 인류 기원 미래 백만 태양계 사라졌던 거인 ...,8,0.1546,"인간, 도시, 인류, 하기, 새로운, 고양이, 그려, 대표, 프랑스, 되고"


In [23]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0      62
1      27
2      48
3      91
4      60
5      50
6      13
7      29
8      31
9      65
10     38
11    105
12     71
13     35
14     48
15     74
16     93
17     60
dtype: int64

In [24]:
df_topic_story[df_topic_story['Dominant_Topic']==1]

,itemId,title,story_sep,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,281536876,파친코 1,어디 하지 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 이민 진의 파친코 면서 끝...,1,0.2880,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
1,281536885,파친코 2,어디 하지 자이니치 분노 슬픔 탄생 대작 한국 세인 미국 이민 진의 파친코 면서 끝...,1,0.2919,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
38,207876451,추사이야기,청소년 읽기 쉽게 풀어서 추사 추사체 완성 일대기 흥미진진 하게 풀어 또한 우리 가...,1,0.0944,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
69,349033022,호랑이를 덫에 가두면,뉴베리상 대상 한국 켈러 아동문학 노벨상 뉴베리상 수상 해님 달님 마법 호랑이 강인...,1,0.1423,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
70,350529382,고구려 7: 동백과 한란,대사 고조선 고구려 몽땅 중국 역사 쓸어 담는 동북공정 허구 깨는 웅혼 고구려 역사...,1,0.6861,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
77,354458230,오백 년째 열다섯,돌이켜 보면 같은 삶은 없었다 새로운 인연 만나면 새로운 시작 우리 신화 옛이야기 ...,1,0.1368,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
117,257461786,뜨거운 피,한국 아르 쌉싸름하면서도 찐득한 간절한 남자 문학동네 캐비닛 프랑스 추리 대상 후보...,1,0.1038,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
123,209729890,칼의 노래,전의 이순신 다시 만나다 동인문학상 수상한 김훈 노래 국가 운명 어진 당대 영웅 정...,1,0.1198,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
164,332181370,"알로하, 나의 엄마들",아프게 기쁘게 뜨겁게 인생 파도 넘어서며 살아갈 여성 펼쳐 내는 가슴 뭉클한 가족 ...,1,0.1606,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
200,286999352,야간비행,신호 언제나 기울였던 생텍쥐페리 명성 안겨 대표 투안 생텍쥐페리 페미 야간비행 비행...,1,0.0922,"여성, 한국, 역사, 새로운, 우리, 인생, 시대, 다섯, 가장, 여자"
